__собираем предложения о продаже домов__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-21_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-22_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-24_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-19_12-40_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

5709


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

5709


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
1376,2253127071,"дом 37,2 м² на участке 7,6 сот.",6200000,,"Межгорская ул., 20","Продается дом в селе Черноречье. Общая площадь дома чуть меньше 40 м. Кв., (жилая площадь 23,1 м. Кв.). Хорошо подойдет для людей с Ипотекой, жаждущие решить свой жилищный вопрос. Площадь участка 7,6 сотки, Ижс. В дальнейшем места хватит для строительства",29,2022-09-15 11:53:29.976,sevastopol,37.20,False,False,False,7.60,6.20,4-8,30-50
22,2327761998,дом 65 м² на участке 4 сот.,6750000,,ул. Штурмана Батурина,"Предлагаем Вашему вниманию прекрасный дом! СРОЧНАЯ ПРОДАЖА!!.\n\nНовый, светлый с прекрасной планировкой. Отлично подойдет для небольшой семьи. Две просторные комнаты, раздельный санузел. Сделан хороший ремонт. В комнатах ламинат, натяжные потолки. Санузел и",1,2022-09-15 11:53:29.976,sevastopol,65.00,False,False,False,4.00,6.75,2-4,50-70
5024,1747027369,дом 114 м² на участке 6 сот.,12950000,,"садоводческое товарищество Дружный, 23","СТ «Дружный». Дом+Баня+Участок Земли. Отличный жилой дом. Газ — Заключен договор о подключении (технологическом присоединении) газоиспользующего оборудования и обьектов капитального строительства к сети газораспредиления. Введен эксплуатацию, регистрация.",47,2022-09-16 15:21:10.419,sevastopol,114.00,False,False,True,6.00,12.95,4-8,70-150


In [11]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5709 entries, 0 to 5708
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5709 non-null   int64         
 1   title                5709 non-null   object        
 2   price                5709 non-null   int64         
 3   obj_name             5709 non-null   object        
 4   adr                  5709 non-null   object        
 5   description          5709 non-null   object        
 6   avito_page           5709 non-null   int64         
 7   ts                   5709 non-null   datetime64[ns]
 8   place                5709 non-null   object        
 9   house_area           5709 non-null   float64       
 10  is_part              5709 non-null   bool          
 11  is_townhouse         5709 non-null   bool          
 12  is_SNT               5709 non-null   bool          
 13  land_area            5709 non-nul

In [14]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [15]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5076,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",48,2022-09-16 15:21:10.419,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
5213,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",1,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
5384,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
5511,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-16 15:24:24.472,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5645,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-19 12:40:34.305,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [16]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

5709


## обновляем таблицу адресов

In [17]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

bahchisaray Бахчисарай,
sudak Судак,
sevastopol Севастополь,


----

In [18]:
from lib.locator import LocationUpdater
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-19 12:42:18 | LocationUpdater: 10187 addresses in location table
[INFO    ] 2022-09-19 12:42:18 | LocationUpdater: 10192 addresses total
[INFO    ] 2022-09-19 12:42:18 | LocationUpdater: 10187 addresses defined
[INFO    ] 2022-09-19 12:42:18 | LocationUpdater: 5 addresses undefined


  0%|          | 0/5 [00:00<?, ?it/s]

[INFO    ] 2022-09-19 12:42:21 | LocationUpdater: 5 new addresses found


In [19]:
loc

,adr,latitude,longitude,truncated
0,"Севастополь, улица Токарева, 7 этап, блок-секция 3",44.78,33.55,False
1,"Севастополь, улица Токарева, 7 этап, блок-секция 1",44.51,33.60,False
2,"Севастополь, улица Генерала Жидилова, 4",44.61,33.52,False
3,"Севастополь, улица Николая Музыки, 96",44.85,33.63,False
4,"Севастополь, улица Лётчиков, д. 10",44.58,33.52,False
...,...,...,...,...
10187,"Судак, Виноградная ул., 1",44.85,34.98,NaN
10188,"Судак, с. Солнечная Долина, Курортный пер., 2",44.86,35.12,NaN
10189,"Судак, с. Миндальное, Миндальная ул., 7",44.86,35.05,NaN
10190,"Судак, Сурожская ул.",44.85,34.99,NaN


In [20]:
loc.sample(3)

,adr,latitude,longitude,truncated
2292,"Севастополь, район Гагаринский, Летчики, улица Колобова, 34/1",44.60,33.46,False
7727,"СНТ Ливада, 57",44.56,33.43,False
1518,"Севастополь, Одесская улица , 25",44.61,33.52,False


In [21]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [22]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 5720
записей без геометки: 0


In [23]:
assert len(data)>0
data.to_pickle(result_file_path)

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5720 entries, 0 to 5719
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             5720 non-null   int64         
 1   title                5720 non-null   object        
 2   price                5720 non-null   int64         
 3   obj_name             5720 non-null   object        
 4   adr                  5720 non-null   object        
 5   description          5720 non-null   object        
 6   avito_page           5720 non-null   int64         
 7   ts                   5720 non-null   datetime64[ns]
 8   place                5720 non-null   object        
 9   house_area           5720 non-null   float64       
 10  is_part              5720 non-null   bool          
 11  is_townhouse         5720 non-null   bool          
 12  is_SNT               5720 non-null   bool          
 13  land_area            5720 non-nul